# Árboles de decisión

En esta clase revisaremos el primer modelo de Machine Learning que veremos, llamado Árbol de decisión. Los Árboles son modelos bastante versátiles y potentes en el mundo del Machine Learning, los cuales presentan las siguientes características:

- Se pueden utilizar para problemas de clasificación, regresión u otros
- No requiere escalar las variables de ingreso
- Alta interpretabilidad
- Forman la base de modelos más sofisticados (Random Forest, entre otros)
 
Para mostrar lo que podemos hacer con un árbol de decisión, utilizaremos un set de datos sobre los precios de inmuebles en la ciudad de Ames, Iowa. 
La base se compone de 2930 registros y contiene un gran número de atributos. 
Nuestro objetivo es generar un modelo que prediga de forma adecuada los precios de inmuebles, medidos con la variable Sale_Price (variable numérica)


In [ ]:
require(tidyverse)
require(tidymodels)
require(Metrics)
require(rpart.plot)
require(rattle)
require(rpart)
require(baguette)

In [ ]:
#Cargamos los datos
ames <- read.csv('~/diplomado_DS_2022/Data/ames_housing.csv')
ames <- ames
head(ames)

Lo primero es ver que tipo de datos son, antes de poder manipular los datos.

In [ ]:
chr_cols <- ames %>%
   select_if(is.character)
chr_cols <- colnames(chr_cols)
ames <- ames %>%
   mutate_each_(funs(factor(.)),chr_cols)
glimpse(ames)

Además de lo anterior, la primera columna es un indicador, y además la latitud y longitud tipicamente no entregan información relevante tal y como están, por lo que las eliminaremos

In [ ]:
ames <- ames %>%
  select(-X,-Longitude,-Latitude)

#Revisando si hay NAs
apply(is.na(ames),2,sum)

Luego sin más rodeos, ajustaremos nuestro modelo de árbol de decisión (en este caso, de regresión). Para ello debemos crear nuestro set de entrenamiento y testeo

In [ ]:
traintest_split <- initial_split(ames, prop = 0.8, strata = NULL)
train_set <- training(traintest_split)
test_set <- testing(traintest_split)
dim(train_set)
dim(test_set)

Para entrenar utilizaremos el workflow de trabajo de `tidymodels` que nos permite entrenar todo tipo de modelos, en particular árboles de decisión, entonces primero veremos un poco como se utiliza. En este caso usaremos la función `decision_tree`que nos permitirá ajustar un árbol de decisión, utilizando distintos motores existentes del modelo. 

In [ ]:
tree_model <- decision_tree(
  mode = "regression", #Aquí podemos usar regression, classification o unknown
  engine = "rpart" #Existen distintos tipos de motores, ya lo veremos
)
tree_model

A un modelo podemos ir modificandolo después de creado sin problema, por ejemplo podemos cambiar el modo

In [ ]:
tree_model <- tree_model %>%
    set_mode('classification')
translate(tree_model)

In [ ]:
tree_model

Ahora tenemos creado un modelo, que será de tipo regresión (variable target continua) y utilizará el motor de ajuste de rpart. Para revisar los distintos motores que se pueden utilizar se utiliza la función `show_engines`

In [ ]:
show_engines('decision_tree')

Ahora, si nos damos cuenta en la parte anterior, hemos creado un modelo pero ¿le entregamos datos? No! en ningún lado le hemos entregado datos al modelo, por lo que si bien hemos creado el modelo, no lo hemos entrenado, que son dos cosas distintas: Primero seleccionamos el modelo a utilizar y luego lo entrenamos, que es lo que veremos ahora. Para entrenar, utilizaremos la funcion `fit`

In [ ]:
tree_fit <- tree_model %>%
    set_mode('regression') %>%
    fit(Sale_Price ~ ., data = train_set)
tree_fit

Lo anterior uno puede entenderlo pero no se ve una ganancia en utilizar esta forma de hacer el modelo, más allá del cambio de motor que utiliza el ajuste. Entonces ahora si, veremos algo que genera un ajuste de modelo basado en recetas, mediante la función `recipe` 

In [ ]:
tree_rec <- recipe(Sale_Price ~ ., data = train_set)

¿Por qué esto es útil? básicamente porque nos permite ajustar distintos modelos (cambiando variables por ejemplo) y/o cambiar el motor utilizado, sin tener que practicamente hacer copy & paste de un codigo e ir modificando. Por otro lado, este tipo de funciones permiten adaptar los roles de las variables (¿roles?). Esto es, cuando ponemos `Sales_price ~ .` significa que Sales_price es el outcome o target, y las demás predictores. Pero si tenemos variables tipo ID? Las podemos eliminar, pero puede ser de utilidad dejarlas para después analizar los resultados.

In [ ]:
tree_rec <- recipe(Sale_Price ~ ., data = train_set) %>%
    update_role(Alley, new_role = 'ID')
summary(tree_rec)

Finalmente, creamos el workflow que nos permite tomar todos los ingredientes anteriores y poder ajustar el modelo

In [ ]:
tree_workflow <- workflow() %>%
    add_model(tree_model) %>%
    add_recipe(tree_rec)
tree_workflow

Y ahora, ajustamos el modelo!

In [ ]:
tree_fit <- tree_workflow %>%
    fit(data = train_set)
tree_fit

In [ ]:
fancyRpartPlot(tree_fit$fit$fit$fit)

In [ ]:
#De manera más elegante
tree_fit_plot <- tree_fit %>% pull_workflow_fit()
fancyRpartPlot(tree_fit_plot$fit)

In [ ]:
pred_values <- tree_fit %>% 
    predict(test_set)
pred_values

In [ ]:
rmse(test_set$Sale_Price, pred_values$.pred)

In [ ]:
R2 <- function(yhat,y){
  rss <- sum((yhat - y)^2)
  tss <- sum((y - mean(y))^2)
  rsq <- 1 - rss/tss
  return(rsq)
}
R2(yhat = pred_values$.pred, y = test_set$Sale_Price)

Ahora existen ciertos hiperparámetros que podemos modificar en los árboles de decisión, de modo de obtener mejores modelos (en sentido de las métricas)

**tree_depth:** Este parámetros determina el largo máximo de las ramas que componen el árbol, desde el nodo inicial al terminal.

**min_n:** Es el número mínimo que debe tener un nodo para poder ser dividido
 
**cost_complexity:** Parámetro de costo o complejidad del modelo (conocido como CP).

In [ ]:
tuned_model <- decision_tree(cost_complexity = tune(), tree_depth = tune()) %>%
    set_engine('rpart') %>%
    set_mode('regression')

tuned_model

La primera forma que exploraremos para hacer tuning de hiper-parámetros es mediante grid search, esto es, generar una grilla de puntos a evaluar, y usaremos las distintas combinaciones para encontrar el mejor modelo para esa grilla.

In [ ]:
tree_grid <- grid_regular(cost_complexity(),
                          tree_depth(),
                          levels = 5)

tree_grid

Tal como lo comentamos en clase, típicamente se utiliza validación cruzada para encontrar nuestros hiper-parámetros, por lo que debemos también configurar aquello.

In [ ]:
set.seed(20220901)
set_folds <- vfold_cv(train_set)

In [ ]:
tree_workflow <- workflow() %>%
    add_model(tuned_model) %>%
    add_formula(Sale_Price ~ .)

In [ ]:
tree_tuning <- tree_workflow %>%
    tune_grid(resamples = set_folds, grid = tree_grid)

In [ ]:
tree_tuning %>%
    collect_metrics()

Podemos graficar los resultados

In [ ]:
tree_tuning %>%
  collect_metrics() %>%
  mutate(tree_depth = factor(tree_depth)) %>%
  ggplot(aes(cost_complexity, mean, color = tree_depth)) +
  geom_line(size = 1.5, alpha = 0.6) +
  geom_point(size = 2) +
  facet_wrap(~ .metric, scales = "free", nrow = 2) +
  scale_x_log10(labels = scales::label_number()) +
  scale_color_viridis_d(option = "plasma", begin = .9, end = 0)

Ahora, ¿cómo elegimos?

In [ ]:
tree_tuning %>%
    show_best('rmse')

Y finalizamos el workflow, para tener nuestro modelos según los parámetros arrojados por el tuning

In [ ]:
best_tree <- tree_tuning %>%
    select_best('rmse')

final_tree_workflow <- tree_workflow %>%
    finalize_workflow(best_tree)

final_tree_workflow

In [ ]:
#ahora ajustamos con los datos
final_tree <- final_tree_workflow %>%
    last_fit(traintest_split)

final_tree %>%
    collect_metrics()

In [ ]:
final_tree %>%
    extract_fit_engine() %>%
    rpart.plot(roundint = FALSE)

Ahora que tenemos un entendimiento de los árboles, abordaremos el concepto de Bagging
   
Este método consiste en generar replicas del dataset de entrenamiento mediante la técnica de bootstrap, de esta forma podremos entrenar nuestro modelo con distintos dataset tomando como predicciones en ellos ya sea un método de votación (en el caso de clasificación) o un promedio (en caso de regresión) entre todos.

La formulación de este método permite claramente generarlo para cualquier modelo, pero el modelo donde más popular se ha hecho este método es utilizandolo sobre árboles de decisión, los que se conocen como Random Forest.

In [ ]:
bag_model <- bag_tree() %>%
    set_engine('rpart') %>%
    set_mode('regression')
bag_model

In [ ]:
#Modelo inicial
bag_wf <- workflow() %>%
    add_model(bag_model) %>%
    add_formula(Sale_Price ~ .) %>%
    fit(data = train_set)

bag_wf


In [ ]:
pred_values <- bag_wf %>% 
    predict(test_set)

rmse(test_set$Sale_Price, pred_values$.pred)
R2(yhat = pred_values$.pred, y = test_set$Sale_Price)

Si nos fijamos, estos resultados ya son mejores (siendo que no hemos hecho tuning de parámetros) que los de nuestro mejor árbol de decisión. Evidentemente estos modelos lo hacen un poco mejor, pero un pierde la capacidad interpretativa del modelo.

# Random Forest 

Lo que estabamos haciendo era seleccionar un conjunto de árboles y generábamos un muestreo de filas para añadir aleatoridad a los árboles. Sin embargo ahora se le agregará un nuevo nivel: se muestrearán ciertos atributos (columnas) para cada nuevo árbol. Así, por ejemplo, el primer árbol puede tener las primeras 3 columnas con las primeras 300 filas, y el segundo árbol puede tener las siguientes columnas, con el resto de filas, generando así independencia entre los modelos.
 
Lo anterior nos permitirá eliminar el sesgo creado por el bagging, pues ya no se está entrenando siempre con todos los atributos.
 
Para ajustar el modelo, utilizaremos el package randomForest
 
Contras:
  - Se pierde la interpretabilidad de un árbol de decisión aislado
 
Pros:
  - Son mas dificiles de sobrajustar
 
La implementación del modelo sigue muy similar al de los anteriores, con la particularidad que el Random Forest, lo haremos paralelizado!!

In [ ]:
require(parallel)
cores <- parallel::detectCores()
cores

Vamos a generar un modelo con tuning de parámetros de manera inmediata

In [ ]:
tuned_rf_model <- rand_forest(mtry = tune(), min_n = tune(), trees = 500) %>%
    set_engine('ranger', num.threads = cores) %>%
    set_mode('regression')

In [ ]:
#Ahora crearemos el workflow
rf_workflow <- workflow() %>%
    add_model(tuned_rf_model) %>%
    add_formula(Sale_Price ~ .)

In [ ]:
rf_model <- rf_workflow %>%
    tune_grid(set_folds,
              grid = 25)

In [ ]:
rf_model %>%
    collect_metrics()

In [ ]:
rf_model %>%
    show_best('rmse')

In [ ]:
best_rf <- rf_model %>%
    select_best('rmse')

final_rf_workflow <- rf_workflow %>%
    finalize_workflow(best_rf)

final_rf <- final_rf_workflow %>%
    last_fit(traintest_split)

final_rf %>%
    collect_metrics()

# XGboots

![kfcv](https://cdn.educba.com/academy/wp-content/uploads/2019/11/bagging-and-boosting.png)

XGboost es un framework  que permite generar un modelo denominado Extreme Gradient Booting decision tree,  que se puede entender como una mejora del random forest. La parte de extreme gradiente es lo que lo hace eficiente en terminos de entrenamiento y costo computaciones, y la parte de booting refiere a la mejora de los métodos anteriores. El modelo de XGBoost funciona similar a un random forest, con la diferencia que en vez de hacerlo cada repetición independiente, va utilizando cada modelo para generado para ir mejorando los que vienen.

Tipicamente tiene permite tener mejores resultados que sus parientes anteriores, y siguiendo el workflow de trabajo que hemos estado utilizando, veremos como ajustar un XGBoost usando tidymodels.

Para esto, utilizaremos otro set de datos, para mostrar también un ejemplo con modelo de clasificación.

In [ ]:
require(nycflights13)

flight_data <- flights %>% 
  mutate(
    arr_delay = ifelse(arr_delay >= 30, "late", "on_time"),
    arr_delay = factor(arr_delay),
    date = lubridate::as_date(time_hour)
  ) %>%
  select(dep_time, flight, origin, dest, air_time, distance, 
         carrier, date, arr_delay) %>% 
  na.omit() %>% 
  mutate_if(is.character, as.factor)

  flight_data

In [ ]:
xgboost_model <- boost_tree() %>%
    set_engine('xgboost') %>%
    set_mode('classification')

In [ ]:
flight_split <- initial_split(flight_data, prop = 0.8, strata = arr_delay)

train_set <- training(flight_split)
test_set <- testing(flight_split)

flight_recipe <- recipe(arr_delay ~ ., data = train_set) %>%
    update_role(flight, date, new_role = 'ID') %>%
    step_date(date, features = c('dow','month')) %>%
    step_dummy(all_nominal_predictors())

In [ ]:
flight_wf <- workflow() %>%
    add_model(xgboost_model) %>%
    add_recipe(flight_recipe)

flight_wf

In [ ]:
flight_fit <- flight_wf %>%
    fit(data =train_set)

Ahora podemos analizar los resultados

In [ ]:
flight_aug <- augment(flight_fit, test_set)
flight_aug

In [ ]:
require(caret)
pred_values <- predict(flight_fit, test_set)
confusionMatrix(pred_values$.pred_class, test_set$arr_delay, positive = 'late')

In [ ]:
flight_aug %>%
    roc_curve(truth = arr_delay, .pred_late) %>%
    autoplot()

flight_aug %>%
    roc_auc(truth = arr_delay, .pred_late)

# Ejercicio

Es sabido que, una entidad que presta servicios o productos (pudiera ser una empresa, un banco, una tienda, etcétera) puede mejorar la experiencia de cliente desarrollando productos personalizados en post de las preferencias y necesidades de cada uno de sus clientes.

El set de datos potencial contiene datos sobre clientes de una institución financiera:

- Customer ID: ID asociado al cliente
- Age: Edad en años del cliente
- Income: Ingreso anual del cliente
- Family: Tamaño del grupo familiar del cliente
- CCAvg: Cupo promedio mensual utilizado en tarjetas de crédito
- Education: Nivel educacional (1 si no es graduado, 2 graduado y 3 si posee estudios especializados (magister, doctorado, etcétera))
- Mortgage: Monto de la hipoteca (0 indica que no posee)
- ZIP Code: Código postal del domicilio
                                
En la última campaña a cada cliente se le ofreció un producto personalizado en base a su comportamiento financiero, preferencias, capacidad de pago y necesidades. La variable target corresponde a Personal Loan el cual indica si el cliente tomó o no tomó este producto (¿El cliente aceptó o no el producto ofrecido?), donde 0 indica que el cliente no adquirió el producto y 1 indica que sí lo adquirió.

Es de interés analizar cuáles pudieran ser los perfiles de clientes que tienen mayor probabilidad de aceptar el producto ofrecido, de manera de, identificar a los clientes con dichas características y priorizarlos a ellos en las próximas campañas.
 
a) Cargue el set de datos. ¿Qué columnas le hacen sentido incluir en un modelo para predecir si un cliente tomará o no el producto ofrecido? Si desea eliminar alguna columna, puede hacerlo según lo visto en clases.

b) Determine cuáles son las variables predictoras que son categorías y modifiquelas si es necesario para poder considerarlas en el modelo. 

c) Determine el set de entrenamiento y testeo en una proposición 80% y 20% respectivamente.

d) Obtenga un árbol de decisión con el set de datos de entrenamiento. Obtenga las métricas pertinentes del modelo en el set de prueba. Muestre el árbol obtenido, ¿qué observa?  ¿cuáles podrían ser los problemas de este árbol? ¿qué alternativas pudieran probarse para abordar este problema?

e) Plantee otro árbol de decisión pero definiendo como parámetro de control la profundidad máxima del árbol. Observe el árbol obtenido. Comente. 

f) Busque mejores valores de los hiperparámetros para este caso, y entregue sus valores. Compare con los resultados anteriores y comente